## Cross-species conservation of motifs

I should write a nice summary and introduction here.

In [19]:
# import everything
import os
import re
import itertools

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.stats import zscore
import gimmemotifs as gimme
from gimmemotifs.motif import read_motifs
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.cluster import FeatureAgglomeration
from sklearn.metrics.cluster import homogeneity_score


import util

In [21]:
# make sure to work with the correct assemblies
if 'snakemake' in globals():
    assemblies = snakemake.params
else:
    import glob
    assemblies = [assemblydir.split("/")[1] for assemblydir in glob.glob("gimme_smooth/**/activity.bayesianridge.score.out.txt")]

# assemblies = ["danRer11", "xenTro9"]
# load the assemblies as dataframes
dfs = dict()
for assembly in assemblies: 
    dfs[assembly] = pd.read_csv(f"gimme_smooth/{assembly}/activity.bayesianridge.score.out.txt", sep="\t", comment="#", index_col=0)

In [22]:
fix, ax = plt.subplots(figsize=(15,15))

df1 = dfs["mm10"]
df2 = dfs["mm10"]
corr = pd.concat([df1, df2], axis=1, keys=['df1', 'df2']).corr().loc['df2', 'df1'].T
*outline, maxima = util.get_outline(corr.values, radius=3)
print(np.sum(maxima))
ax.plot(*outline, color=(1,0,0,.5), linewidth=3)
ax.matshow(corr.values)
for (i, j), z in np.ndenumerate(corr.values):
    ax.text(j, i, '{:0.2f}'.format(z), ha='center', va='center')

21


In [30]:
nr_assemblies = len(dfs.keys())

fig, axes = plt.subplots(figsize=(30, 30), nrows=len(dfs.keys()), ncols=len(dfs.keys()))
# fig, axes = plt.subplots(figsize=(15, 15), nrows=len(dfs.keys()), ncols=len(dfs.keys()))
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.05, hspace=0.05)

early = pd.DataFrame(index=dfs["danRer11"].index)
late = pd.DataFrame(index=dfs["danRer11"].index)

def get_bars(assembly):
    bars = dict()
    for phase, color in colors.items():
        start = timings.loc[assembly, phase]
        col_idx = list(timings.columns).index(phase)
        if col_idx != -1:
            for end_col in timings.columns[col_idx + 1:]:
                end = timings.loc[assembly, end_col]
                if not np.isnan(end):
                    bars[phase] = (start, end, color)
                    break
    return bars

for assembly_1_i, assembly_1 in enumerate(dfs.keys()):
    for assembly_2_i, assembly_2 in enumerate(dfs.keys()):
        ax = axes[assembly_1_i, assembly_2_i]

        if assembly_2_i > assembly_1_i: 
            ax.axis('off')
            continue
        df1 = dfs[assembly_1]
        df2 = dfs[assembly_2]
        corr = pd.concat([df1, df2], axis=1, keys=['df1', 'df2']).corr().loc['df2', 'df1'].T

        if assembly_1 == assembly_2:
            vmin, vmax = 0, 1
            cmap = "Blues"
        else:
            vmin, vmax = 0, 0.25
            cmap = "pink_r"
                
        im = ax.matshow(corr.values, aspect=len(df2.columns)/len(df1.columns), vmin=vmin, vmax=vmax, cmap=cmap)
        if assembly_1 != assembly_2:
            *outline, maxima = util.get_outline(corr.values, radius=3)
            ax.plot(*outline, color=(1,0,0,.5), linewidth=3)
            if len(np.where(maxima)[0]) == 2:
                time_1_1, time_1_2 = np.where(maxima)[0]
                time_2_1, time_2_2 = np.where(maxima)[1]
                
#                 df_tmp[:] = zscore(df_tmp, axis=0)
                df_early = pd.concat([df1.iloc[:, time_1_1], df2.iloc[:, time_2_1]], axis=1)
                df_early["average"] = np.mean(df_early.values, axis=1)
                early[f"{assembly_1} & {assembly_2}"] = df_early["average"].rank(method='average', ascending=False)

                df_late = pd.concat([df1.iloc[:, time_1_2], df2.iloc[:, time_2_2]], axis=1)
                df_late["average"] = np.mean(df_late.values, axis=1)
                late[f"{assembly_1} & {assembly_2}"] = df_late["average"].rank(method='average', ascending=False)

        ax.set_xticks([])
        ax.set_yticks([])
        if assembly_2_i == 0:
#             ax.set_yticks(range(corr.shape[0]), minor=False)
#             ax.set_yticklabels(corr.index, fontsize=14)
            ax.set_ylabel(util.ass2name(assembly_1), fontsize=18)
            
#             for start, stop, color in get_bars(assembly_1).values():
#                 p_start = float(corr.index[0])
#                 p_stop = float(corr.index[-1])
#                 stop  = list(sorted([0, 1, (stop - p_start) / (p_stop - p_start)]))[1]
#                 start = list(sorted([0, 1, (start  - p_start) / (p_stop - p_start)]))[1]
#                 add_bar_left(im, start, stop, color=color)

        if assembly_1_i == nr_assemblies - 1:
            ax.set_xlabel(util.ass2name(assembly_2), fontsize=18)
#             ax.set_xticks(range(corr.shape[1]), minor=False)
#             ax.set_xticklabels(corr.columns, fontsize=14)
#             ax.xaxis.set_ticks_position("bottom")
#             for start, stop, color in get_bars(assembly_2).values():
#                 p_start = float(corr.columns[0])
#                 p_stop = float(corr.columns[-1])
#                 stop  = list(sorted([0, 1, (stop - p_start) / (p_stop - p_start)]))[1]
#                 start = list(sorted([0, 1, (start  - p_start) / (p_stop - p_start)]))[1]
#                 add_bar_lower(im, start, stop, color=color)
        
#         # pretty colorbar
#         divider = make_axes_locatable(ax)
#         cax = divider.append_axes("right", size="5%", pad=0.1)
#         plt.colorbar(im, cax=cax)

#         ax.set_aspect('equal')
# fig.colorbar(im, ax=axes[0])

In [23]:
early["mean"] = early.mean(axis=1)
late["mean"] = late.mean(axis=1)
early = early.sort_values("mean")
late = late.sort_values("mean")

In [24]:
import IPython
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    IPython.display.display(early.sort_values("mean").head(10))


,mm10 & Astyanax_mexicanus-2.0,GRCg6a & Astyanax_mexicanus-2.0,GRCg6a & mm10,danRer11 & Astyanax_mexicanus-2.0,ASM223467v1 & Astyanax_mexicanus-2.0,ASM223467v1 & mm10,ASM223467v1 & GRCg6a,ASM223467v1 & danRer11,Bl71nemr & Astyanax_mexicanus-2.0,Bl71nemr & mm10,Bl71nemr & GRCg6a,Bl71nemr & danRer11,Bl71nemr & ASM223467v1,xenTro9 & Astyanax_mexicanus-2.0,xenTro9 & mm10,xenTro9 & GRCg6a,xenTro9 & danRer11,xenTro9 & ASM223467v1,xenTro9 & Bl71nemr,ASM318616v1 & Astyanax_mexicanus-2.0,ASM318616v1 & mm10,ASM318616v1 & danRer11,ASM318616v1 & ASM223467v1,ASM318616v1 & Bl71nemr,ASM318616v1 & xenTro9,mean
GM.5.0.Homeodomain_POU.0007,3.0,8.0,479.0,2.0,2.0,26.0,49.0,11.0,1.0,7.0,6.0,2.0,3.0,1.0,63.0,25.0,1.0,1.0,1.0,2.0,85.0,3.0,7.0,2.0,1.0,31.64
GM.5.0.Homeodomain_POU.0022,97.0,67.0,305.0,17.0,10.0,21.0,46.0,19.0,7.0,63.0,52.0,7.0,6.0,3.0,4.0,6.0,3.0,4.0,3.0,11.0,214.0,18.0,10.0,7.0,4.0,40.16
GM.5.0.Sox.0028,11.0,4.0,50.0,13.0,5.0,5.0,6.0,5.0,2.0,1.0,84.0,9.0,1.0,2.0,205.0,7.0,4.0,2.0,2.0,4.0,662.0,6.0,2.0,1.0,3.0,43.84
GM.5.0.Homeodomain_POU.0020,9.0,28.0,242.0,7.0,8.0,28.0,44.0,15.0,4.0,57.0,86.0,5.0,9.0,6.0,530.0,119.0,6.0,10.0,5.0,8.0,172.0,9.0,13.0,6.0,7.0,57.32
GM.5.0.C2H2_ZF.0108,2.0,5.0,101.0,29.0,1.0,4.0,5.0,2.0,5.0,372.0,729.0,214.0,4.0,9.0,361.0,41.0,22.0,6.0,10.0,1.0,149.0,8.0,1.0,4.0,6.0,83.64
GM.5.0.Unknown.0094,57.0,40.0,118.0,51.0,18.0,15.0,14.0,28.0,97.0,646.0,605.0,233.0,38.0,22.0,34.0,17.0,33.0,15.0,55.0,13.0,180.0,20.0,9.0,41.0,15.0,96.56
GM.5.0.Sox.0023,166.0,56.0,58.0,139.0,22.0,9.0,10.0,16.0,26.0,4.0,581.0,104.0,12.0,32.0,240.0,45.0,29.0,17.0,21.0,70.0,1068.0,83.0,15.0,31.0,23.0,115.08
GM.5.0.C2H2_ZF.0010,68.0,19.0,39.0,41.0,11.0,7.0,7.0,8.0,154.0,727.0,829.0,118.0,16.0,61.0,151.0,27.0,77.0,22.0,154.0,38.0,38.0,33.0,12.0,155.0,80.0,115.68
GM.5.0.T-box.0001,102.0,38.0,146.0,9.0,55.0,637.0,109.0,66.0,13.0,675.0,34.0,4.0,14.0,44.0,367.0,79.0,40.0,103.0,27.0,27.0,459.0,15.0,112.0,10.0,78.0,130.52
GM.5.0.T-box.0003,13.0,47.0,628.0,5.0,15.0,264.0,171.0,31.0,8.0,497.0,360.0,18.0,32.0,15.0,275.0,182.0,20.0,29.0,31.0,20.0,266.0,39.0,137.0,108.0,74.0,131.40


In [7]:
import IPython
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    IPython.display.display(late.sort_values("mean")["mean"].head(10))

GM.5.0.C2H2_ZF.0097                    32.64
GM.5.0.RFX.0015                        98.56
GM.5.0.Nuclear_receptor.0069          105.56
GM.5.0.Homeodomain_Paired_box.0002    119.60
GM.5.0.RFX.0006                       127.76
GM.5.0.Nuclear_receptor.0140          134.64
GM.5.0.C2H2_ZF.0094                   140.24
GM.5.0.Unknown.0167                   177.72
GM.5.0.Unknown.0065                   193.36
GM.5.0.Homeodomain.0093               200.52
Name: mean, dtype: float64

In [8]:
def fake_bootstrap(nr_motifs, nr_columns, tries=1_000):
    """
    
    """
    bootstrapped = np.zeros((nr_motifs, tries))
    fake_ranks = np.array([[rank for rank in range(nr_motifs)] for col in range(nr_columns)]).T

    for i in range(tries):
        # randomize our fake ranks
        for column in range(nr_columns):
            np.random.shuffle(fake_ranks[:, column])

        bootstrapped[:, i] = -np.sort(np.mean(-fake_ranks, axis=1))[::-1]
    return bootstrapped

# now bootstrap
result = fake_bootstrap(early.shape[0], early.shape[1] - 1)
result_sorted = np.sort(result, axis=1)

In [9]:
total_tries = result_sorted.shape[1]

ys_real = early["mean"]
xs = [i for i in range(len(ys_real))]


# not really confidence interval, but comes close
confidence_ish = 0.005, 0.995
confidence_ish = 0, 1

fig = plt.Figure()
ax = plt.gca()


ax.fill_between(xs, 
                result_sorted[:, int(total_tries * confidence_ish[0])], 
                result_sorted[:, int(total_tries * confidence_ish[1]) - 1], 
               alpha=0.5)
plt.scatter(xs, ys_real, c='r', s=0.2)
plt.axvline(x=100)
plt.axvline(x=len(xs) - 100)

In [10]:
from gimmemotifs.c_metrics import pfmscan, score
from gimmemotifs.comparison import RCDB
motifs = read_motifs()

motif_scores = dict()
for motif in motifs:
    motif_scores[motif] = pfmscan(RCDB, motif.pwm, motif.pwm_min_score(), len(RCDB), False, True)



In [11]:
# pd.DataFrame.from_dict(motifs)
# motif_scores["GM.5.0.Sox.0001_AACAAT"]

In [12]:
arr = np.array(list(motif_scores.values()))

<ipython-input-12-503166603521>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr = np.array(list(motif_scores.values()))


In [13]:
print(arr.shape)

(1796,)


In [14]:
for key, value in motif_scores.items():
    print(len(value))

8187
8182
8182
8174
8185
8179
8188
8178
8183
8177
8181
8182
8187
8187
8186
8181
8182
8174
8182
8181
8169
8183
8181
8187
8183
8185
8188
8180
8185
8180
8182
8183
8184
8187
8184
8185
8184
8184
8180
8182
8183
8183
8176
8184
8183
8177
8186
8187
8186
8186
8184
8180
8185
8183
8172
8179
8184
8186
8184
8188
8176
8185
8185
8179
8182
8184
8186
8185
8182
8185
8186
8186
8180
8182
8186
8178
8184
8185
8186
8185
8179
8187
8184
8187
8185
8182
8183
8183
8187
8180
8179
8183
8178
8186
8182
8183
8185
8179
8173
8185
8185
8183
8181
8182
8185
8184
8183
8183
8177
8179
8185
8184
8180
8187
8186
8184
8174
8178
8184
8189
8178
8185
8183
8184
8179
8167
8172
8188
8184
8183
8187
8187
8187
8186
8185
8180
8182
8180
8183
8187
8178
8186
8182
8187
8175
8172
8184
8184
8187
8179
8185
8185
8183
8185
8185
8182
8187
8185
8184
8185
8188
8179
8186
8183
8180
8184
8184
8176
8173
8187
8185
8187
8176
8188
8186
8183
8181
8185
8187
8184
8187
8179
8183
8165
8184
8180
8182
8188
8184
8184
8173
8184
8187
8179
8185
8187
8172
8180
8188
8185


In [15]:
# load the assemblies as dataframes
dfs_scores = dict()
for assembly in assemblies: 
    dfs[assembly] = pd.read_csv(f"gimme_smooth/{assembly}/motif.score.txt.gz", sep="\t", comment="#", index_col=0, compression='gzip')

all_scores = pd.concat(dfs.values())

In [16]:
# # get the labels
# dbd_re = "(?:GM\.5\.0\.)(.+)(?:\.\d{4}.*)"
# true_groups = [re.search(dbd_re, motif.__str__()).group(1) for motif in motifs]
# group2label = {group: i for i, group in enumerate(set(true_groups))}
# true_labels = [group2label[re.search(dbd_re, group).group(1)] for group in all_scores.columns]

# p = PCA(n_components=100)
# pcs = pd.DataFrame(p.fit_transform(all_scores.T), index=all_scores.columns)
# # xs = list(range(3, len(all_scores.columns) - 1))
# xs = np.linspace(len(group2label), len(all_scores.columns) - 1, 100)
# ys = {
#     "FeatureAgglomeration": [],
#     ""
# }
# homogeneity_scores = []
# for x in xs:
#     x = int(x)
#     fa = FeatureAgglomeration(
#                 n_clusters=x, affinity="euclidean", linkage="complete",
#     )
#     fa.fit(pcs.T)
#     homogen_score = homogeneity_score(true_labels, fa.labels_)
#     homogeneity_scores.append(homogen_score)
#     sil_score = silhouette_score(all_scores.T, fa.labels_)
#     ys.append(sil_score)
    

SyntaxError: invalid syntax (<ipython-input-16-736039df2632>, line 14)

In [ ]:
# print(sil_score)
print(max(ys))
print(ys.count(max(ys)))
print(xs[ys.index(max(ys))])

In [ ]:
# plt.plot(xs, ys)
plt.plot(xs, homogeneity_scores)
# plt.axvline(xs[ys.index(max(ys))])

In [ ]:
plt.plot(xs, ys)

In [ ]:
plt.plot(xs, np.array(ys) * np.array(homogeneity_scores))

In [ ]:
nr_clusters = 600

p = PCA(n_components=100)
pcs = pd.DataFrame(p.fit_transform(all_scores.T), index=all_scores.columns)
fa = FeatureAgglomeration(
            n_clusters=nr_clusters, affinity="euclidean", linkage="complete",
)
fa.fit(pcs.T)




In [ ]:
random_cluster = 106
idxs = [i for i, label in enumerate(fa.labels_) if label==random_cluster]

for i, idx in enumerate(idxs):
    
    
    motifs[idx].plot_logo(add_left=2)